In [1]:
import pandas as pd
import numpy as np

N = 500000

def inverse_logit(x):
    return 1. / (1. + np.exp(-x))

u = np.random.normal(size=N)
x = np.random.binomial(1, p=inverse_logit(u))
z = np.random.binomial(1, p=0.25 + 0.5*x)
y = np.random.binomial(1, p=0.5*z + 0.5*inverse_logit(u))

df = pd.DataFrame({'X': x, 'Z': z, 'Y': y})

In [2]:
# true ATE is 0.5 * 0.5 = 0.25
df.groupby('X').mean()['Y'][1] - df.groupby('X').mean()['Y'][0]

0.3376959217685063

We want to estimate $P(Y|do(X=x)) = \sum_z P(Z=z|X=x) \sum_{x'}P(Y|X=x', Z=z)P(X=x')$

This has three components:

1. $ P(Z=z|X=x)$
2. $P(Y|X=x', Z=z)$, and
3. $P(X=x')$

First, $ P(Z=z|X=x)$, using the trick that $E[Z] = P(Z)$ for binary $Z$

In [38]:
df.groupby('X').mean()['Z']

X
0    0.247669
1    0.750254
Name: Z, dtype: float64

then $P(Y|X=x', Z=z)$

In [39]:
df.groupby(['X', 'Z']).mean()['Y']

X  Z
0  0    0.206088
   1    0.703770
1  0    0.291150
   1    0.793087
Name: Y, dtype: float64

and finally $P(X=x')$

In [40]:
df.mean()[['X']], 1. - df.mean()[['X']]

(X    0.500292
 dtype: float64,
 X    0.499708
 dtype: float64)

putting it all together,

In [3]:
def p_y_given_do_x(y, x, df=df):
    result = 0.
    for z in df.Z.unique():
        for xp in df.X.unique():
            f1 = z * df.groupby('X').mean()['Z'][x] + (1-z)* (1-df.groupby('X').mean()['Z'][x])
            f2 = y * df.groupby(['X', 'Z']).mean()['Y'][xp, z] + (1-y)*(1-df.groupby(['X', 'Z']).mean()['Y'][xp, z])
            f3 = x * df.mean()[['X']] + (1-x)*(1-df.mean()[['X']])
            result += f1 * f2 * f3
    return result

In [4]:
p_y_given_do_x(1, 1) - p_y_given_do_x(1, 0)

X    0.251098
dtype: float64